In [ ]:
# Classifying forams in paleo oceanography Kaggle competition for ICS 637 @University of Hawaii.

In [ ]:
# Download data files from public google drive.
# Takes ~1min
!pip install --upgrade --no-cache-dir gdown
import gdown

file_ids = {'train.zip': '1mm2tEB05wQwNHP0SySTCp1-BL6G1IHf0',
            'test.zip': '1Cf-yAfSt706w10p5Dij7ppFLo6Se8Ej7',
            'train.csv': '1NAd3UPTWWxmXdJ9N_Mr3GWgMmO_Aojnn',
            'test.csv': '1Koi9hpUuUwn1swel9QLsEOr_MZKX2LOg',
            }

for output, file_id in file_ids.items():
  url=f"https://drive.google.com/uc?id={file_id}"
  gdown.download(url, output, quiet=False)

!unzip -q train.zip
!unzip -q test.zip


Downloading...
From (original): https://drive.google.com/uc?id=1mm2tEB05wQwNHP0SySTCp1-BL6G1IHf0
From (redirected): https://drive.google.com/uc?id=1mm2tEB05wQwNHP0SySTCp1-BL6G1IHf0&confirm=t&uuid=c81819db-9b69-40a9-9059-fb6ef3e3af8a
To: /content/train.zip
100%|██████████| 75.2M/75.2M [00:01<00:00, 42.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Cf-yAfSt706w10p5Dij7ppFLo6Se8Ej7
To: /content/test.zip
100%|██████████| 18.9M/18.9M [00:00<00:00, 44.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NAd3UPTWWxmXdJ9N_Mr3GWgMmO_Aojnn
To: /content/train.csv
100%|██████████| 283k/283k [00:00<00:00, 78.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Koi9hpUuUwn1swel9QLsEOr_MZKX2LOg
To: /content/test.csv
100%|██████████| 65.6k/65.6k [00:00<00:00, 45.8MB/s]


In [ ]:
#Import Libraries & Process Data

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import pandas as pd


class ImageCSVDataset(Dataset):
    def __init__(self, image_dir, csv_file, transform=None, test_set=False):
        """
        Args:
            image_dir (string): Directory with all the images.
            csv_file (string): Path to the csv file with labels.
            transform (callable, optional): Optional transform to be applied
                on a sample.

        """
        self.image_dir = image_dir
        self.labels = pd.read_csv(csv_file)
        self.transform = transform
        self.test_set = test_set

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.labels.iloc[idx]['filename']  # Assuming image name is in the first column
        image = Image.open(f"{self.image_dir}/{img_name}").convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.test_set:
          return image

        label = self.labels.iloc[idx]['label']  # Assuming label is in the second column
        return image, label

# Define dataset and data loader.
image_dir = 'train/'
csv_file = 'train.csv'
image_dir_test = 'test/'
csv_file_test = 'test.csv'

# Define transformations (optional)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=45),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.01),  # Add Gaussian noise
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.225, 0.225, 0.225])
])

dataset = ImageCSVDataset(image_dir=image_dir, csv_file=csv_file, transform=transform)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Split dataset: 90% for training, 10% for validation
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader for validation set:
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# Import & Finetune the Resnet18 CNN Model:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load Resnet18
model = models.resnet18(pretrained=True)

# Get num classes
num_classes = dataset.labels['label'].max()+1  # Get number of unique classes from the dataset
print(num_classes)

# Modify the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)  # Replace with your number of classes

# Freeze early layers (optional) (when false they are frozen)
for param in model.parameters():
    param.requires_grad = False


# Unfreeze only the first layer
#first_layer = list(model.children())[0]  # Get the first layer
#for param in first_layer.parameters():
#    param.requires_grad = True


# Unfreeze later layers (optional)
for layer in list(model.children())[-10:]:  # Unfreeze last 3 layers
    for param in layer.parameters():
        param.requires_grad = True

# 4. Define optimizer and loss function
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss()

# 5. Create DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 6. Train the model (similar to previous example)
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

best_val_loss = np.inf


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # Training loop
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 100 == 99:
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Training Loss: {running_loss / 100:.3f}")
            running_loss = 0.0

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(val_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Save the model if validation loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Validation loss decreased, saving model with loss: {best_val_loss:.4f}")

    # Step the scheduler to decrease learning rate if needed
    scheduler.step()

print(f"Finished Training with best validation loss: {best_val_loss:.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


54
[Epoch 1, Batch 100] Training Loss: 1.529
[Epoch 1, Batch 200] Training Loss: 0.842
Epoch 1/20, Validation Loss: 0.6736, Validation Accuracy: 80.48%
Validation loss decreased, saving model with loss: 0.6736
[Epoch 2, Batch 100] Training Loss: 0.685
[Epoch 2, Batch 200] Training Loss: 0.665
Epoch 2/20, Validation Loss: 0.6141, Validation Accuracy: 82.10%
Validation loss decreased, saving model with loss: 0.6141
[Epoch 3, Batch 100] Training Loss: 0.570
[Epoch 3, Batch 200] Training Loss: 0.606
Epoch 3/20, Validation Loss: 0.5037, Validation Accuracy: 83.95%
Validation loss decreased, saving model with loss: 0.5037
[Epoch 4, Batch 100] Training Loss: 0.536
[Epoch 4, Batch 200] Training Loss: 0.537
Epoch 4/20, Validation Loss: 0.4640, Validation Accuracy: 84.76%
Validation loss decreased, saving model with loss: 0.4640
[Epoch 5, Batch 100] Training Loss: 0.480
[Epoch 5, Batch 200] Training Loss: 0.495
Epoch 5/20, Validation Loss: 0.4465, Validation Accuracy: 86.49%
Validation loss decr

In [ ]:
#Save model with other name (optional)
torch.save(model.state_dict(), 'best_model1.pth')

In [ ]:
# Load best model (optional)

model.load_state_dict(torch.load('best_model.pth'))  # Load the saved model parameters
model.to(device)

<ipython-input-10-9b529d19e15d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))  # Load the saved model parameters


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Use model to make predictions.

model.eval()  # Set to evaluation mode


# Create test dataloader that doesn't have labels.
dataset_test = ImageCSVDataset(image_dir=image_dir_test, csv_file=csv_file_test,
                               transform=transform, test_set=True)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

predictions = []
for i, data in enumerate(dataloader_test, 0):
    inputs = data.to(device)
    outputs = model(inputs)
    predictions.append(outputs.argmax(axis=1).detach().cpu().numpy())
predictions = np.concatenate(predictions)

# Write predictions to a submission file.
df_predictions = pd.read_csv(csv_file_test)
df_predictions['predictions'] = predictions
df_predictions[['id', 'predictions']].to_csv('submission.csv', index=False)
